In [18]:
import cv2 
import numpy as np

In [41]:
threshold = 0.5 
suppression_threshold = 0.7 
ssd_input_size = 320

In [22]:
'''
DetectionModel(modelPath, configPath)
modelPath  -> includes the weights of the model
configPath -> defines the architecture of the model
'''
neural_network = cv2.dnn_DetectionModel('ssd_weights.pb', 'ssd_mobilenet_coco_cfg.pbtxt')

In [24]:
'''
The strip() method removes any leading and trailing whitespaces.
Leading means at the beginning of the string, trailing means at the end.
rstrip() -> removes any trailing whitespaces.
lstrip() -> removes any leading whitespaces.
strip(character) -> a set of characters to remove as leading/trailing characters.
'''
def construct_class_names(file_name = 'class_names'):
    with open(file_name, "rt") as file:
        names = file.read().rstrip().split('\n')
    return names

In [43]:
def show_detected_objects(image, bboxes_keep, all_bounding_boxes, object_names, class_ids):
    for index in bboxes_keep:
        box = all_bounding_boxes[index]
        x, y, w, h = box[0], box[1], box[2], box[3]
        cv2.rectangle(image, (x, y), (x+w, y+h), color = (0, 255, 0), thickness = 2)
        cv2.putText(image, object_names[class_ids[index]-1].upper(), (x, y-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 
               fontScale = 0.7, color = (0, 255, 0), thickness = 1)

In [47]:
neural_network.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
neural_network.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
neural_network.setInputSize(ssd_input_size, ssd_input_size)
neural_network.setInputScale(1.0 / 127.5) # normalization between [-1, 1]
neural_network.setInputMean((127.5, 127.5, 127.5))
neural_network.setInputSwapRB(True)

class_names = construct_class_names('class_names')

capture = cv2.VideoCapture('objects.mp4')

while capture.isOpened():
    
    is_grabbed, frame = capture.read()

    if not is_grabbed:
        break

    class_label_ids, confidences, bboxes = neural_network.detect(frame)
    ''' 
        returns 3 values after tracking each frame.
        class_label_ids = (N,1)
        confidences     = (N,1)
        bboxes          = (N,4) (x, y, w, h)
    '''
    bboxes = list(bboxes)
    confidences = np.array(confidences).flatten()

    boxes_to_keep = cv2.dnn.NMSBoxes(bboxes, confidences, threshold, suppression_threshold)

    show_detected_objects(frame, boxes_to_keep, bboxes, class_names, class_label_ids)

    cv2.imshow('SSD Algorithm', frame)
    cv2.waitKey(1)

capture.release()
cv2.destroyAllWindows()